# Master Discogs Database

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-08-21 19:45:17.509347


In [2]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4177 artist entries
Counter({'Discogs': 4010, 'AllMusic': 3952, 'MusicBrainz': 3743, 'LastFM': 2860, 'DatPiff': 536, 'RockCorner': 481, 'AceBootlegs': 176, 'CDandLP': 173, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4010, 'AllMusic': 3952, 'MusicBrainz': 3743, 'LastFM': 2860, 'DatPiff': 536, 'RockCorner': 481, 'AceBootlegs': 176, 'CDandLP': 173, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})


In [3]:
maindb = mainDB(mdb=mdb, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
maindb.setDBFull()
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RockCorner
  Creating Database Records for DatPiff
  Creating Database Records for AceBootlegs
  Creating Databas

/Volumes/Piggy/Discog/artists-discogs-db/metadata/92-Metadata.p  	92 698355
/Volumes/Piggy/Discog/artists-discogs-db/metadata/93-Metadata.p  	93 705854
/Volumes/Piggy/Discog/artists-discogs-db/metadata/94-Metadata.p  	94 713360
/Volumes/Piggy/Discog/artists-discogs-db/metadata/95-Metadata.p  	95 720925
/Volumes/Piggy/Discog/artists-discogs-db/metadata/96-Metadata.p  	96 728354
/Volumes/Piggy/Discog/artists-discogs-db/metadata/97-Metadata.p  	97 735856
/Volumes/Piggy/Discog/artists-discogs-db/metadata/98-Metadata.p  	98 743290
/Volumes/Piggy/Discog/artists-discogs-db/metadata/99-Metadata.p  	99 750818



Saving 750818 entries to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p

Saving data to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.7MB.
Saved data to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.7MB.
Saving 750818 entries to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToName.p

Saving data to /Users/tgadfor

/Volumes/Piggy/Discog/artists-discogs-db/metadata/6-MediaMetadata.p 	427149    931158    3994281   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/60-MediaMetadata.p 	434670    948086    4069338   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/61-MediaMetadata.p 	442146    965482    4141567   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/62-MediaMetadata.p 	449731    982155    4209325   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/63-MediaMetadata.p 	457311    997900    4280199   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/64-MediaMetadata.p 	464946    1014272   4349683   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/65-MediaMetadata.p 	472503    1031764   4424508   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/66-MediaMetadata.p 	480018    1048338   4494938   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/67-MediaMetadata.p 	487625    1064110   4563520   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/68-MediaMetadata.p 	494987    1082457   46

/Volumes/Piggy/Discog/artists-allmusic-db/metadata/12-Metadata.p  	23 12301
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/20-Metadata.p  	24 12812
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/27-Metadata.p  	25 13331
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/6-Metadata.p  	26 13838
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/13-Metadata.p  	27 14347
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/28-Metadata.p  	28 14875
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/42-Metadata.p  	29 15360
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/35-Metadata.p  	30 15869
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/49-Metadata.p  	31 16432
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/43-Metadata.p  	32 16931
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/29-Metadata.p  	33 17444
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/36-Metadata.p  	34 17933
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/50-Metadata.p  	35 18510
/Volumes/Pigg

/Volumes/Piggy/Discog/artists-allmusic-db/metadata/15-MediaMetadata.p 	2535      10603     29545     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/22-MediaMetadata.p 	3032      12838     35413     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/1-MediaMetadata.p 	3508      14893     41037     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/8-MediaMetadata.p 	4020      17165     46637     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/16-MediaMetadata.p 	4537      19446     53659     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/23-MediaMetadata.p 	5098      22036     60863     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/2-MediaMetadata.p 	5631      23998     67129     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/9-MediaMetadata.p 	6159      26572     73804     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/24-MediaMetadata.p 	6690      28787     79744     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/17-MediaMetadata.p 	7211      31376

...

	Shape --> (4977, 5)
DataFrame Shape is (4977, 5)
Current Time is Fri Aug 21, 2020 19:57:48 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 79.0 millseconds.
Saving Master Artist DB File: /Users/tgadfort/Music/Discog/db-cdandlp/MasterSlimArtistDB.p
  --> This file is 713.1kB.
Current Time is Fri Aug 21, 2020 19:57:48 for Creating Artist DBs
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/0-MediaMetadata.p 	64        409       409       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/1-MediaMetadata.p 	128       866       866       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/10-MediaMetadata.p 	189       1349      1349      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/11-MediaMetadata.p 	234       1735      1735      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/12-MediaMetadata.p 	276       

/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/82-MediaMetadata.p 	4046      32177     32177     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/83-MediaMetadata.p 	4097      32474     32474     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/84-MediaMetadata.p 	4146      32871     32871     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/85-MediaMetadata.p 	4199      33295     33295     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/86-MediaMetadata.p 	4252      33677     33677     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/87-MediaMetadata.p 	4297      33976     33976     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/88-MediaMetadata.p 	4343      34275     34275     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/89-MediaMetadata.p 	4396      34752     34752     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/9-MediaMetadata.p 	4445      35201     35201     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/90-MediaMetadata.p 	4506      35659     35

/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/18-MediaMetadata.p 	8         0         478       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/19-MediaMetadata.p 	9         0         484       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/20-MediaMetadata.p 	10        0         486       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/21-MediaMetadata.p 	11        0         566       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/22-MediaMetadata.p 	12        0         589       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/23-MediaMetadata.p 	13        0         619       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/26-MediaMetadata.p 	14        0         657       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/27-MediaMetadata.p 	15        0         737       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/28-MediaMetadata.p 	17        0         840       
/Volumes/Piggy/Discog/artists-rateyou

/Volumes/Piggy/Discog/artists-musicstack-db/metadata/29-Metadata.p  	19 50
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/30-Metadata.p  	20 52
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/31-Metadata.p  	21 53
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/34-Metadata.p  	22 54
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/36-Metadata.p  	23 57
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/37-Metadata.p  	24 61
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/38-Metadata.p  	25 63
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/4-Metadata.p  	26 65
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/40-Metadata.p  	27 66
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/41-Metadata.p  	28 67
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/42-Metadata.p  	29 70
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/43-Metadata.p  	30 73
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/44-Metadata.p  	31 75
/Volumes/Piggy/Discog/arti

/Volumes/Piggy/Discog/artists-musicstack-db/metadata/51-MediaMetadata.p 	85        0         188       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/52-MediaMetadata.p 	86        0         189       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/53-MediaMetadata.p 	87        0         192       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/54-MediaMetadata.p 	90        0         209       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/55-MediaMetadata.p 	95        0         381       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/57-MediaMetadata.p 	96        0         384       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/58-MediaMetadata.p 	100       0         393       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/59-MediaMetadata.p 	103       0         397       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/6-MediaMetadata.p 	105       0         399       
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/60-MediaMeta

In [ ]:
maindb = mainDB(mdb=mdb, create=False, debug=True)

In [ ]:
maindb.setDBFull()

# Metadata

In [ ]:
%load_ext autoreload
%autoreload

from multiprocessing import Pool
from mainDB import mainDB
import time
mainDB = mainDB()
dbdata = mainDB.dbdata

def parseArtistsAM(modVal, force=False, doExtra=False):
    dbdata["AllMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=True, doExtra=False):
    dbdata["Discogs"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    dbdata["MusicBrainz"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    dbdata['DatPiff']['Artists'].parseArtistFiles()
    #artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    dbdata["RateYourMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    dbdata["LastFM"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    dbdata["RockCorner"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRC.parseArtistModValFiles(modVal, force=force)

def parseArtistsCL(modVal, force=False, doExtra=False):
    dbdata["CDandLP"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsCL.parseArtistModValFiles(modVal, force=force)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        dbdata["RateYourMusic"]["Artists"].parseDownloadedFiles()
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

In [ ]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
parseArtistsParallel(db="Discogs", nProcs=4)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
#parseArtistsParallel(db="LastFM", nProcs=3)
#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['LastFM']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)